# **2. nn.Module로 구현하는 로지스틱 회귀**

## 2.1 파이토치의 nn.Linear와 nn.Sigmoid로 로지스틱 회귀 구현하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [64]:
# 훈련 데이터
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

nn.Sequential()은 nn.Module 층을 차례로 쌓을 수 있도록 합니다. 쉽게 말해 nn.Sequential()은 $Wx+b$와 같은 수식과 시그모이드 함수 등과 같은 여러 함수들을 연결해주는 역할을 합니다.

In [65]:
model = nn.Sequential(
    nn.Linear(2, 1), # input_dim = 2, output_dim = 1
    nn.Sigmoid() # 출력은 sigmoid 함수를 거친다.
)

model(x_train)

tensor([[0.1939],
        [0.0903],
        [0.0971],
        [0.0311],
        [0.0180],
        [0.0142]], grad_fn=<SigmoidBackward0>)

현재 $W$와 $b$는 랜덤 초기화가 된 상태입니다. 그렇기에 현재 예측값은 의미가 없습니다. 이를 의미있게 만들기 위해 경사 하강법을 진행하면

In [66]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
        epoch, nb_epochs, cost.item(), accuracy * 100,
    ))

Epoch    0/1000 Cost: 2.026148 Accuracy 50.00%
Epoch   10/1000 Cost: 0.562432 Accuracy 50.00%
Epoch   20/1000 Cost: 0.437817 Accuracy 66.67%
Epoch   30/1000 Cost: 0.370633 Accuracy 83.33%
Epoch   40/1000 Cost: 0.314211 Accuracy 83.33%
Epoch   50/1000 Cost: 0.264154 Accuracy 83.33%
Epoch   60/1000 Cost: 0.218761 Accuracy 100.00%
Epoch   70/1000 Cost: 0.180787 Accuracy 100.00%
Epoch   80/1000 Cost: 0.156604 Accuracy 100.00%
Epoch   90/1000 Cost: 0.143658 Accuracy 100.00%
Epoch  100/1000 Cost: 0.133904 Accuracy 100.00%
Epoch  110/1000 Cost: 0.125447 Accuracy 100.00%
Epoch  120/1000 Cost: 0.118013 Accuracy 100.00%
Epoch  130/1000 Cost: 0.111428 Accuracy 100.00%
Epoch  140/1000 Cost: 0.105553 Accuracy 100.00%
Epoch  150/1000 Cost: 0.100279 Accuracy 100.00%
Epoch  160/1000 Cost: 0.095520 Accuracy 100.00%
Epoch  170/1000 Cost: 0.091202 Accuracy 100.00%
Epoch  180/1000 Cost: 0.087267 Accuracy 100.00%
Epoch  190/1000 Cost: 0.083665 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080357 Accuracy 100.00

정확도가 올라가고 어느 순간부턴 실제값과 일치한 결과를 내는 것을 볼 수 있다. 훈련 후, $W$와 $b$를 출력해보면 앞 챕터와 거의 비슷한 것을 확인할 수 있다.

In [67]:
list(model.parameters())

[Parameter containing:
 tensor([[3.2538, 1.5183]], requires_grad=True),
 Parameter containing:
 tensor([-14.4856], requires_grad=True)]

## 2.2 인공 신경망으로 표현되는 로지스틱 회귀

사실 로지스틱 회귀는 인공 신경망으로 간주할 수 있다.

<center>
<img src="https://wikidocs.net/images/page/58686/logistic_regression.PNG"/>
</center>

위의 그림에서 각 화살표는 입려고가 곱해지는 가중치(편향)다. 각 입력 $x$는 각 입력의 가중치 $W$와 곱해지고 편향 $b$는 상수 1과 곱해지는 것으로 표현되었다. 그리고 출력하기 전에 시그모이드 함수를 지나게 된다. 

결과적으로 위의 인공 신경망은 다음과 같은 다중 로지스틱 회귀를 표현하고 있다.

$$H(x) = sigmoid(x_1w_1+x_2w_2+b)$$

---
### 참고) 시그모이드 함수는 인공 신경망의 은닉층에는 거의 사용되지 않는다.